* Sequential
    - 딥러닝의 구조를 한 층 한 층 쉽게 쌓아올릴수 있게 해줌
    - Sequential 함수 선언 후 model.add() 함수를 사용해 필요한 층을 차례로 추가해주면 됨
* Dense
    - model.add() 함수에는 Dense() 함수가 포함되어 있음 
    - dense는 '조밀하게 모여있는 집합' 이라는 뜻으로 각 층이 제각각 어떤 특성을 가질지 옵션을 설정하는 역할을 함
    - 딥러닝의 구조와 층별 옵션을 정하고 나면 compile() 함수를 사용해 이를 실행 시킴
    - 예문
        ```python
        model.add(Dense(30, input_dim=17, activation='relu'))
        model.add(Dense(1,activation='sigmoid'))
        model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])    
        ```
    - 추후 나올 키워드들 미리 설명
        1. activation
            + 다음 층으로 어떻게 값을 넘길지 결정하는 부분
            + relu와 sigmoid 함수를 자주 사용함
        2. loss
            + 한번 신경망이 실행될 때마다 오차 값을 추적하는 함수
        3. optimizer
            + 오차를 어떻게 줄여 나갈지 정하는 함수


# 1. 선형회귀

## 1.1 선형회귀 정의
* 구하고자 하는 값을 변화하게 하는 __요소__를 x라하고 이 x 값에 의해 변화하는 __구하고자 하는 값__을 y라 정의 해봄
* 수식으로 나타내면 y = ax + b 여기서 a는 기울기 b는 절편
* 여기서 x를 __독립변수__ y를 __종속변수__ 라고함
* x가 하나면 __단순 선형회귀__(simple linear regression)
* x가 여러개면 __다중 선형회귀__(multiple linear regression)

### 1.2 최소 제곱법(method of least squares)
* 선형회귀에서 목표는 __가장 정확한 직선__을 긋는 것
* a = (x-x평균)(y-y평균)의 합 / (x-x평균)제곱의 합
* b = y의 평균 - (x의 평균 * 기울기 a)

In [1]:
# -*- coding: utf-8 -*-
import numpy as np

# x 값과 y값
x=[2, 4, 6, 8]
y=[81, 93, 91, 97]

# x와 y의 평균값
mx = np.mean(x)
my = np.mean(y)
print("x의 평균값:", mx)
print("y의 평균값:", my)

# 기울기 공식의 분모
divisor = sum([(mx - i)**2 for i in x])

# 기울기 공식의 분자
def top(x, mx, y, my):
    d = 0
    for i in range(len(x)):
        d += (x[i] - mx) * (y[i] - my)
    return d
dividend = top(x, mx, y, my)

print("분모:", divisor)
print("분자:", dividend)

# 기울기와 y 절편 구하기
a = dividend / divisor
b = my - (mx*a)

# 출력으로 확인
print("기울기 a =", a)
print("y 절편 b =", b)

x의 평균값: 5.0
y의 평균값: 90.5
분모: 20.0
분자: 46.0
기울기 a = 2.3
y 절편 b = 79.0


## 1.2 평균 제곱근 오차(root mean square error)
* 독립변수가 하나일때는 최소 제곱법을 쓰면 되지만 여러개일때는 무작위로 값을 대입해야함
* 독립변수가 두개 이상일 경우의 기울기와 b를 찾아낼때 오차를 조금씩 수정하기 위한 오차 평가 알고리즘
* 실제값 - 예측값을 해서 오차값을 구한 후 부호제거를 위해 제곱한 후 평균을 구함. __평균 제곱 오차__(Mean Squared Error, MSE)
* 이 평균 제곱 오차에 ROOT값을 씌우면 평균 제곱근 오차가 됨

In [3]:
import numpy as np

#기울기 a와 y 절편 b
ab=[3,76]

# x,y의 데이터 값
data = [[2, 81], [4, 93], [6, 91], [8, 97]]
x = [i[0] for i in data]
y = [i[1] for i in data]

# y=ax + b에 a,b 값 대입하여 결과를 출력하는 함수
def predict(x):
   return ab[0]*x + ab[1]

# RMSE 함수
def rmse(p, a):
   return np.sqrt(((p - a) ** 2).mean())

# RMSE 함수를 각 y값에 대입하여 최종 값을 구하는 함수
def rmse_val(predict_result,y):
   return rmse(np.array(predict_result), np.array(y))

# 예측값이 들어갈 빈 리스트
predict_result = []

# 모든 x값을 한 번씩 대입하여 predict_result 리스트완성.
for i in range(len(x)):
   predict_result.append(predict(x[i]))
   print("공부시간=%.f, 실제점수=%.f, 예측점수=%.f" % (x[i], y[i], predict(x[i])))

# 최종 RMSE 출력
print("rmse 최종값: " + str(rmse_val(predict_result,y)))

공부시간=2, 실제점수=81, 예측점수=82
공부시간=4, 실제점수=93, 예측점수=88
공부시간=6, 실제점수=91, 예측점수=94
공부시간=8, 실제점수=97, 예측점수=100
rmse 최종값: 3.3166247903554


# 2. 오차 수정하기: 경사 하강법

## 2.1 경사 하강법 개요
* a를 무한대로 키우면 오차도 무한대로 커지고
* a를 무한대로 작게 해도 오차가 무한대로 커짐
* 적절한 a를 구하는게 필요함
* 이때 미분을 이용하여 순간기울기가 0인 지점을 찾는것이 목적
* 즉 경사하강법의 순서는 아래와 같음
    1. a1에서 미분을 구함
    2. 구해진 기울기의 반대방향(기울기가 + 면 음의 방향, -면 양의방향)으로 얼마간 이동시킨 a2에서 미분을 구함
    3. a3에서 미분을 구함
    4. 3의 값이 0이 아니면 위의 과정을 반복함
    
## 2.2 학습률(learning rate)
* 기울기의 부호를 바꿔 이동시킬 때 이동 거리를 정해주는 것
* 케라스는 학습률을 자동으로 조절해준다함

In [4]:

import tensorflow as tf

# x, y의 데이터 값
data = [[2, 81], [4, 93], [6, 91], [8, 97]]
x_data = [x_row[0] for x_row in data]
y_data = [y_row[1] for y_row in data]

# 기울기 a와 y 절편 b의 값을 임의로 정한다.
# 단, 기울기의 범위는 0 ~ 10 사이이며 y 절편은 0 ~ 100 사이에서 변하게 한다.
a = tf.Variable(tf.random_uniform([1], 0, 10, dtype = tf.float64, seed = 0))
b = tf.Variable(tf.random_uniform([1], 0, 100, dtype = tf.float64, seed = 0))

# y에 대한 일차 방정식 ax+b의 식을 세운다.
y = a * x_data + b

# 텐서플로 RMSE 함수
rmse = tf.sqrt(tf.reduce_mean(tf.square( y - y_data )))

# 학습률 값
learning_rate = 0.1

# RMSE 값을 최소로 하는 값 찾기
gradient_decent = tf.train.GradientDescentOptimizer(learning_rate).minimize(rmse)

# 텐서플로를 이용한 학습
with tf.Session() as sess:
    # 변수 초기화
    sess.run(tf.global_variables_initializer())
    # 2001번 실행(0번 째를 포함하므로)
    for step in range(2001):
        sess.run(gradient_decent)
        # 100번마다 결과 출력
        if step % 100 == 0:
            print("Epoch: %.f, RMSE = %.04f, 기울기 a = %.4f, y 절편 b = %.4f" % (step,sess.run(rmse),sess.run(a),sess.run(b)))

Instructions for updating:
Colocations handled automatically by placer.
Epoch: 0, RMSE = 30.2139, 기울기 a = 7.5235, y 절편 b = 80.5984
Epoch: 100, RMSE = 2.8860, 기울기 a = 2.2299, y 절편 b = 79.4181
Epoch: 200, RMSE = 2.8826, 기울기 a = 2.2601, y 절편 b = 79.2379
Epoch: 300, RMSE = 2.8815, 기울기 a = 2.2773, y 절편 b = 79.1353
Epoch: 400, RMSE = 2.8811, 기울기 a = 2.2871, y 절편 b = 79.0770
Epoch: 500, RMSE = 2.8810, 기울기 a = 2.2927, y 절편 b = 79.0438
Epoch: 600, RMSE = 2.8810, 기울기 a = 2.2958, y 절편 b = 79.0249
Epoch: 700, RMSE = 2.8810, 기울기 a = 2.2976, y 절편 b = 79.0142
Epoch: 800, RMSE = 2.8810, 기울기 a = 2.2987, y 절편 b = 79.0081
Epoch: 900, RMSE = 2.8810, 기울기 a = 2.2992, y 절편 b = 79.0046
Epoch: 1000, RMSE = 2.8810, 기울기 a = 2.2996, y 절편 b = 79.0026
Epoch: 1100, RMSE = 2.8810, 기울기 a = 2.2998, y 절편 b = 79.0015
Epoch: 1200, RMSE = 2.8810, 기울기 a = 2.2999, y 절편 b = 79.0008
Epoch: 1300, RMSE = 2.8810, 기울기 a = 2.2999, y 절편 b = 79.0005
Epoch: 1400, RMSE = 2.8810, 기울기 a = 2.3000, y 절편 b = 79.0003
Epoch: 1500, RMSE = 2.88

# 3. 다중 선형 회귀
* 이전에 봤던 선형회귀는 독립변수를 하나만 사용했지만 얜 두개 이상임

In [5]:
import tensorflow as tf

# x1, x2, y의 데이터 값

data = [[2, 0, 81], [4, 4, 93], [6, 2, 91], [8, 3, 97]]
x1 = [x_row1[0] for x_row1 in data]
x2 = [x_row2[1] for x_row2 in data] # 새로 추가되는 값
y_data = [y_row[2] for y_row in data]

# 기울기 a와 y절편 b의 값을 임의로 정함. 단 기울기의 범위는 0-10 사이, y 절편은 0-100사이에서 변하게 함
a1 = tf.Variable(tf.random_uniform([1], 0, 10, dtype=tf.float64, seed=0))
a2 = tf.Variable(tf.random_uniform([1], 0, 10, dtype=tf.float64, seed=0))
b = tf.Variable(tf.random_uniform([1], 0, 100, dtype=tf.float64, seed=0))

# 새로운 방정식
y = a1 * x1 + a2 * x2+ b

# 텐서플로 RMSE 함수
rmse = tf.sqrt(tf.reduce_mean(tf.square( y - y_data )))

# 학습률 값
learning_rate = 0.1

# RMSE 값을 최소로 하는 값 찾기
gradient_decent = tf.train.GradientDescentOptimizer(learning_rate).minimize(rmse)

# 학습이 진행되는 부분
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2001):
        sess.run(gradient_decent)
        if step % 100 == 0:
            print("Epoch: %.f, RMSE = %.04f, 기울기 a1 = %.4f, 기울기 a2 = %.4f, y절편 b = %.4f" % (step,sess.run(rmse),sess.run(a1),sess.run(a2),sess.run(b)))


Epoch: 0, RMSE = 49.1842, 기울기 a1 = 7.5270, 기울기 a2 = 7.8160, y절편 b = 80.5980
Epoch: 100, RMSE = 1.8368, 기울기 a1 = 1.1306, 기울기 a2 = 2.1316, y절편 b = 78.5119
Epoch: 200, RMSE = 1.8370, 기울기 a1 = 1.1879, 기울기 a2 = 2.1487, y절편 b = 78.1057
Epoch: 300, RMSE = 1.8370, 기울기 a1 = 1.2122, 기울기 a2 = 2.1571, y절편 b = 77.9352
Epoch: 400, RMSE = 1.8370, 기울기 a1 = 1.2226, 기울기 a2 = 2.1607, y절편 b = 77.8636
Epoch: 500, RMSE = 1.8370, 기울기 a1 = 1.2269, 기울기 a2 = 2.1622, y절편 b = 77.8335
Epoch: 600, RMSE = 1.8370, 기울기 a1 = 1.2288, 기울기 a2 = 2.1628, y절편 b = 77.8208
Epoch: 700, RMSE = 1.8370, 기울기 a1 = 1.2295, 기울기 a2 = 2.1631, y절편 b = 77.8155
Epoch: 800, RMSE = 1.8370, 기울기 a1 = 1.2299, 기울기 a2 = 2.1632, y절편 b = 77.8133
Epoch: 900, RMSE = 1.8370, 기울기 a1 = 1.2300, 기울기 a2 = 2.1632, y절편 b = 77.8124
Epoch: 1000, RMSE = 1.8370, 기울기 a1 = 1.2301, 기울기 a2 = 2.1633, y절편 b = 77.8120
Epoch: 1100, RMSE = 1.8370, 기울기 a1 = 1.2301, 기울기 a2 = 2.1633, y절편 b = 77.8118
Epoch: 1200, RMSE = 1.8370, 기울기 a1 = 1.2301, 기울기 a2 = 2.1633, y절편 b = 77.81

# 4. 로지스틱 회귀
* 종속변수가 명목형(Yes, No 같이 수치형이 아님)일때 사용하는 방법